**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Linux namespaces

![Namespaces](namespaces.png)

![container vs hypervisor](docker-hypervisor.png)

In [10]:
docker ps

CONTAINER ID   IMAGE                  COMMAND                  CREATED              STATUS              PORTS     NAMES
ea714724541c   nginx                  "/docker-entrypoint.…"   48 seconds ago       Up 47 seconds                 k8s_nginx_nginx-6799fc88d8-nzbj2_default_2c126dba-6a37-46be-b550-d0e0d8a30bf9_0
afd2530f876d   k8s.gcr.io/pause:3.5   "/pause"                 About a minute ago   Up About a minute             k8s_POD_nginx-6799fc88d8-nzbj2_default_2c126dba-6a37-46be-b550-d0e0d8a30bf9_0
612d18f433bd   6e38f40d628d           "/storage-provisioner"   32 minutes ago       Up 32 minutes                 k8s_storage-provisioner_storage-provisioner_kube-system_b147672e-36fd-42d8-a691-2c3896f0f43b_0
65793eecdde3   8d147537fb7d           "/coredns -conf /etc…"   32 minutes ago       Up 32 minutes                 k8s_coredns_coredns-78fcd69978-jtj8x_kube-system_e01e0c8b-f513-4822-983a-264e8326eeb1_0
f915b870ee6b   k8s.gcr.io/pause:3.5   "/pause"                 32 minutes ago      

In [3]:
crictl ps

CONTAINER           IMAGE               CREATED             STATE               NAME                      ATTEMPT             POD ID
612d18f433bdd       6e38f40d628db       29 minutes ago      Running             storage-provisioner       0                   f915b870ee6b8
65793eecdde34       8d147537fb7d1       29 minutes ago      Running             coredns                   0                   ed853a4bf5d00
72c6ee947db8d       873127efbc8a7       29 minutes ago      Running             kube-proxy                0                   cfa9996f98c4c
b3473922bbe4c       5425bcbd23c54       29 minutes ago      Running             kube-controller-manager   0                   b2787fd720749
fc75fd7aba150       0048118155842       29 minutes ago      Running             etcd                      0                   286180e64eaa5
3cab011b29341       b51ddc1014b04       29 minutes ago      Running             kube-scheduler            0                   fd8093a82c02b
c305e0932e5a9       e64579b

In [5]:
kubectl create deployment nginx --image=nginx

deployment.apps/nginx created


In [7]:
kubectl get all

NAME                         READY   STATUS    RESTARTS   AGE
pod/nginx-6799fc88d8-nzbj2   1/1     Running   0          21s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   32m

NAME                    READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/nginx   1/1     1            1           21s

NAME                               DESIRED   CURRENT   READY   AGE
replicaset.apps/nginx-6799fc88d8   1         1         1       21s


In [11]:
POD=k8s_nginx_nginx-6799fc88d8-nzbj2_default_2c126dba-6a37-46be-b550-d0e0d8a30bf9_0

In [12]:
STATE_PID=$(docker inspect --format '{{.State.Pid}}' $POD)
echo Enter
echo nsenter -t $STATE_PID -n bash
echo in a terminal  

Enter
nsenter -t 11165 -n bash
in a terminal


In [13]:
nsenter --help


Usage:
 nsenter [options] [<program> [<argument>...]]

Run a program with namespaces of other processes.

Options:
 -a, --all              enter all namespaces
 -t, --target <pid>     target process to get namespaces from
 -m, --mount[=<file>]   enter mount namespace
 -u, --uts[=<file>]     enter UTS namespace (hostname etc)
 -i, --ipc[=<file>]     enter System V IPC namespace
 -n, --net[=<file>]     enter network namespace
 -p, --pid[=<file>]     enter pid namespace
 -C, --cgroup[=<file>]  enter cgroup namespace
 -U, --user[=<file>]    enter user namespace
 -S, --setuid <uid>     set uid in entered namespace
 -G, --setgid <gid>     set gid in entered namespace
     --preserve-credentials do not touch uids or gids
 -r, --root[=<dir>]     set the root directory
 -w, --wd[=<dir>]       set the working directory
 -F, --no-fork          do not fork before exec'ing <program>
 -Z, --follow-context   set SELinux context according to --target PID

 -h, --help             display this help
 -V

In [14]:
nsenter -t $STATE_PID -n ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
5: eth0@if6: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:ac:11:00:03 brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 172.17.0.3/16 brd 172.17.255.255 scope global eth0
       valid_lft forever preferred_lft forever


In [15]:
ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
2: docker0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:88:4a:25:70 brd ff:ff:ff:ff:ff:ff
    inet 172.17.0.1/16 brd 172.17.255.255 scope global docker0
       valid_lft forever preferred_lft forever
4: vetha322d0f@if3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue master docker0 state UP group default 
    link/ether 12:a2:0c:e3:69:21 brd ff:ff:ff:ff:ff:ff link-netnsid 1
6: veth2027c2e@if5: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue master docker0 state UP group default 
    link/ether 8e:f1:eb:30:c1:ce brd ff:ff:ff:ff:ff:ff link-netnsid 2
24: eth0@if25: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:c0:a8:31:02 brd ff:ff

 # Danger!
 
 OpenVpn works and could be used as backdoor

In [16]:
nsenter -t $STATE_PID -a hostname

nginx-6799fc88d8-nzbj2


In [17]:
nsenter -t $STATE_PID -a ip a

nsenter: failed to execute ip: No such file or directory


: 127

In [18]:
nsenter -t $STATE_PID -n ip route

default via 172.17.0.1 dev eth0 
172.17.0.0/16 dev eth0 proto kernel scope link src 172.17.0.3 


In [19]:
ip route

default via 192.168.49.1 dev eth0 
172.17.0.0/16 dev docker0 proto kernel scope link src 172.17.0.1 
192.168.49.0/24 dev eth0 proto kernel scope link src 192.168.49.2 


In [20]:
nsenter -t $STATE_PID -a ls

bin   docker-entrypoint.d   home   media  proc	sbin  tmp
boot  docker-entrypoint.sh  lib    mnt	  root	srv   usr
dev   etc		    lib64  opt	  run	sys   var


In [21]:
ls

 AccountWithRBAC.ipynb       OpenPolicyAgent.ipynb
 AuditLogs.ipynb             OpenShift
 ClusterOverview.ipynb       Overview.ipynb
 ContainerdCloudbomb.ipynb   PodSecurityPolicyBasic.ipynb
 CouchDB.ipynb               PodSecurityPolicyPermissive.ipynb
 CrioBomb.ipynb              PodSecurityPolicyStrict.ipynb
 Harbor.ipynb               'Postgres Zalando.ipynb'
 Helm.ipynb                  README.md
 Ingress.ipynb               ServiceAccountToken.ipynb
 IngressKong.ipynb           Unprivileged.ipynb
 IngressKongStart.ipynb      Untitled.ipynb
 IngressStart.ipynb          default-rbac.png
 Istio-Traffic-Mgmt.ipynb    defaut-rbac.dot
 Istio1.8.2.ipynb            docker-hypervisor.png
 IstioHack.ipynb             kube-bench.ipynb
 Kube-Scan-Cloudbomb.ipynb   minikube.dot
 KubernetesBasics.ipynb      minikube.png
 License.ipynb               my-namespace-rbac.dot
'Linux Namespaces.ipynb'     my-namespace-rbac.png
 Makefile                    namespaces.dot
 NetworkPolicy.ipynb         

In [22]:
nsenter -t $STATE_PID -a ps auxwww

nsenter: failed to execute ps: No such file or directory


: 127

In [23]:
ps auxww

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0  22332 11620 ?        Ss   09:04   0:01 /sbin/init
root         176  0.0  0.0  27052 10808 ?        S<s  09:04   0:00 /lib/systemd/systemd-journald
message+     188  0.0  0.0   7020  3680 ?        Ss   09:04   0:00 /usr/bin/dbus-daemon --system --address=systemd: --nofork --nopidfile --systemd-activation --syslog-only
root         192  0.1  0.2 1640740 46768 ?       Ssl  09:04   0:02 /usr/bin/containerd
root         197  0.0  0.0  12184  7448 ?        Ss   09:04   0:00 sshd: /usr/sbin/sshd -D [listener] 0 of 10-100 startups
root         221  0.4  0.5 341128 82440 ?        Ssl  09:04   0:09 /usr/bin/python3 /usr/local/bin/jupyter-lab --ip 192.168.49.2 --allow-root --log-level=INFO
root         465  1.8  0.6 1974280 96784 ?       Ssl  09:04   0:41 /usr/bin/dockerd -H tcp://0.0.0.0:2376 -H unix:///var/run/docker.sock --default-ulimit=nofile=1048576:1048576 --tlsverify --tlscacert /etc/doc

In [24]:
nsenter -t $STATE_PID -p ps auxwww

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0  22332 11620 ?        Ss   09:04   0:01 /sbin/init
root         176  0.0  0.0  27052 10808 ?        S<s  09:04   0:00 /lib/systemd/systemd-journald
message+     188  0.0  0.0   7020  3680 ?        Ss   09:04   0:00 /usr/bin/dbus-daemon --system --address=systemd: --nofork --nopidfile --systemd-activation --syslog-only
root         192  0.1  0.2 1640740 46768 ?       Ssl  09:04   0:02 /usr/bin/containerd
root         197  0.0  0.0  12184  7448 ?        Ss   09:04   0:00 sshd: /usr/sbin/sshd -D [listener] 0 of 10-100 startups
root         221  0.4  0.5 341448 82732 ?        Ssl  09:04   0:10 /usr/bin/python3 /usr/local/bin/jupyter-lab --ip 192.168.49.2 --allow-root --log-level=INFO
root         465  1.8  0.6 1974280 96784 ?       Ssl  09:04   0:41 /usr/bin/dockerd -H tcp://0.0.0.0:2376 -H unix:///var/run/docker.sock --default-ulimit=nofile=1048576:1048576 --tlsverify --tlscacert /etc/doc